`The the goal of this file is to prepare dataset for analysis by performing initial exploration, cleaning, and preprocessing.`

# 1. Loading dataset:

In [5]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
df = pd.read_csv('../data/Walmart.csv')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


# 2. Initial Exploration:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


### Converting Date feature to datetype

In [7]:
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

In [9]:
df.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6435,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,2011-06-17 00:00:00,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
min,1.000000,2010-02-05 00:00:00,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,2010-10-08 00:00:00,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,2011-06-17 00:00:00,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,2012-02-24 00:00:00,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,2012-10-26 00:00:00,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000
std,12.988182,NaN,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885


#### Basic Observations:

1. Store: There are 45 total stores
2. Date: The date range from 02 Feb 2010 to 26 Oct 2012 (Total of 994 days)
3. Weekly Sales:
    - The lowest sale was on 03 Dec 2010 in Store no. 33 of $209,986.25
    - The highest sale was on 24 Dec 2010 in Store no. 14 of $3,818,686.45
    - The average sales were $1,046,964.88
4. Temperature:
    - The lowest temperature was on 04 Feb 2011 of -2.06 F at Store no. 7
    - The highest temperature was on 16 Jul 2010 of 100.14  F at Store no. 33
    - The average temperature was 60.66 F
5. Fuel Price:
    - The lowest fuel price was on 19 Feb 2010 of $2.472 per gallon
    - The highest fuel price was on 12 Oct 2012 of $4.468 per gallon
    - The average fuel price was of $3.356 per gallon
6. CPI:
    - The lowest CPI was on 13 Aug 2010 of 126.064
    - The highest CPI was on 26 Oct 2012 of 227.23
    - The average CPI was 171.58
7. Unemployment:
    - The lowest unemployment rate was in Oct 2012 of 3.879%
    - The highest unemployment rate were in Oct, Nov and Dec of 2010 of 14.313%
    - The average unemployment rate was 8%

In [27]:
df['Holiday_Flag'].nunique()

2

In [28]:
df['Holiday_Flag'].value_counts()

Holiday_Flag
0    5985
1     450
Name: count, dtype: int64

#### Creating a new column `Holiday_Event` to specific which holiday is a certain date if it's a holiday else not a holiday.

- Holiday Events: <br /> 
    - Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13<br /> 
    - Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13<br /> 
    - Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13<br /> 
    - Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

**These dates are not the exact dates of those events, rather the week date that's present in the data.**

In [51]:
# map of events and their dates
holiday_map = {
    "Super Bowl Week": [dt.datetime(2010, 2, 12), dt.datetime(2011, 2, 11), dt.datetime(2012, 2, 10)],
    "Labour Day Week": [dt.datetime(2010, 9, 10), dt.datetime(2011, 9, 9), dt.datetime(2012, 9, 7)],
    "Thanksgiving Week": [dt.datetime(2010, 11, 26), dt.datetime(2011, 11, 25), dt.datetime(2012, 11, 23)],
    "Christmas Week": [dt.datetime(2010, 12, 31), dt.datetime(2011, 12, 30), dt.datetime(2012, 12, 28)]
}

# holiday lookup table
holiday_lookup = {date: event for event, dates in holiday_map.items() for date in dates}

df['Holiday_Event'] = [holiday_lookup.get(date, "Not a holiday") for date in df['Date']]

In [52]:
df['Holiday_Event'].value_counts()

Holiday_Event
Not a holiday        5985
Super Bowl Week       135
Labour Day Week       135
Thanksgiving Week      90
Christmas Week         90
Name: count, dtype: int64

#### Creating separate cols for day, month and year

In [53]:
df['day'] = df['Date'].dt.day   # Extracting day from Date
df['month'] = df['Date'].dt.month # Extracting month from Date
df['year'] = df['Date'].dt.year   # Extracting year from Date

In [54]:
df['day'].nunique(), df['month'].nunique(), df['year'].nunique()    # Unique values for day, month, year

(31, 12, 3)

# 3. Handling Missing and duplicate values:

In [55]:
df.isna().sum()  # Check for missing values

Store            0
Date             0
Weekly_Sales     0
Holiday_Flag     0
Temperature      0
Fuel_Price       0
CPI              0
Unemployment     0
Holiday_Event    0
day              0
month            0
year             0
dtype: int64

Thankfully there are no missing values in the data.

In [56]:
df.duplicated().sum()  # Check for duplicate rows

np.int64(0)

There are no duplicate rows either.

#### Why does Labour day and Super Bowl appears more times than any other holiday events?

- The data is from 02 Feb 2010 to 26 Oct 2012, so Thanksgiving and Christmas weeks appears only twice - in 2010 and 2011, and not in 2012. Therefore, 45 stores * 2 years = 90 appearances.

- Whereas Super Bowl and Labour Day appears in 2012 as well, thus 45*3 = 135 counts.

In [60]:
print("Super Bowl dates:- ", df[df['Holiday_Event'] == "Super Bowl Week"]['Date'].unique())
print("\nLabour day dates:- ", df[df['Holiday_Event'] == "Labour Day Week"]['Date'].unique())
print("\nThanksgiving dates:- ", df[df['Holiday_Event'] == "Thanksgiving Week"]['Date'].unique())
print("\nChristmas dates:- ", df[df['Holiday_Event'] == "Christmas Week"]['Date'].unique())

Super Bowl dates:-  <DatetimeArray>
['2010-02-12 00:00:00', '2011-02-11 00:00:00', '2012-02-10 00:00:00']
Length: 3, dtype: datetime64[ns]

Labour day dates:-  <DatetimeArray>
['2010-09-10 00:00:00', '2011-09-09 00:00:00', '2012-09-07 00:00:00']
Length: 3, dtype: datetime64[ns]

Thanksgiving dates:-  <DatetimeArray>
['2010-11-26 00:00:00', '2011-11-25 00:00:00']
Length: 2, dtype: datetime64[ns]

Christmas dates:-  <DatetimeArray>
['2010-12-31 00:00:00', '2011-12-30 00:00:00']
Length: 2, dtype: datetime64[ns]


In [65]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Event,day,month,year
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,Not a holiday,5,2,2010
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,Super Bowl Week,12,2,2010
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,Not a holiday,19,2,2010
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,Not a holiday,26,2,2010
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,Not a holiday,5,3,2010


# Storing cleaned data for further analysis:

In [66]:
df.to_csv('../data/01_data_cleaned.csv', index=False)   # storing cleaned data